In [2]:
pip install mysql-connector

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pandas

     |████████████████████████████████| 9.7 MB 1.5 MB/s eta 0:00:01     |█                               | 286 kB 1.5 MB/s eta 0:00:07     |█████████████▎                  | 4.0 MB 5.2 MB/s eta 0:00:02
     |████████████████████████████████| 15.4 MB 12.7 MB/s eta 0:00:01   |█████████████████▏              | 8.2 MB 1.5 MB/s eta 0:00:05     |██████████████████████▋         | 10.9 MB 1.5 MB/s eta 0:00:03     |█████████████████████████▉      | 12.4 MB 12.7 MB/s eta 0:00:01     |███████████████████████████▍    | 13.1 MB 12.7 MB/s eta 0:00:01     |█████████████████████████████   | 13.9 MB 12.7 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
from sqlalchemy import create_engine
source = create_engine('mysql+mysqlconnector://test:test123@db/test')

In [7]:
import pandas as pd

pd.read_sql('select current_date', con=source)

,current_date
0,2021-02-04


In [8]:
df = pd.read_csv('country_vaccinations.csv', 
                 dtype={"total_vaccinations": "Int64", 
                        "people_vaccinated": "Int64", 
                        "people_fully_vaccinated": "Int64", 
                        "daily_vaccinations": "Int64", 
                        "daily_vaccinations_per_million": "Int64"}, encoding='utf-8')
df

,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website
0,Argentina,ARG,12/29/20,700,<NA>,<NA>,NaN,<NA>,0.00,NaN,NaN,<NA>,Sputnik V,Ministry of Health,http://datos.salud.gob.ar/dataset/vacunas-cont...
1,Argentina,ARG,12/30/20,<NA>,<NA>,<NA>,NaN,15656,NaN,NaN,NaN,346,Sputnik V,Ministry of Health,http://datos.salud.gob.ar/dataset/vacunas-cont...
2,Argentina,ARG,12/31/20,32013,<NA>,<NA>,NaN,15656,0.07,NaN,NaN,346,Sputnik V,Ministry of Health,http://datos.salud.gob.ar/dataset/vacunas-cont...
3,Argentina,ARG,1/1/21,<NA>,<NA>,<NA>,NaN,11070,NaN,NaN,NaN,245,Sputnik V,Ministry of Health,http://datos.salud.gob.ar/dataset/vacunas-cont...
4,Argentina,ARG,1/2/21,<NA>,<NA>,<NA>,NaN,8776,NaN,NaN,NaN,194,Sputnik V,Ministry of Health,http://datos.salud.gob.ar/dataset/vacunas-cont...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1422,Wales,NaN,1/19/21,176186,175816,370,13989.0,10672,5.59,5.58,0.01,3385,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/health...
1423,Wales,NaN,1/20/21,190831,190435,396,14645.0,11105,6.05,6.04,0.01,3522,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/health...
1424,Wales,NaN,1/21/21,212732,212317,415,21901.0,12318,6.75,6.73,0.01,3907,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/health...
1425,Wales,NaN,1/22/21,241016,240547,469,28284.0,15148,7.64,7.63,0.01,4804,"Oxford/AstraZeneca, Pfizer/BioNTech",Government of the United Kingdom,https://coronavirus.data.gov.uk/details/health...


In [9]:
df.loc[df.source_website == 'Kuwait', ['source_website']]

,source_website


In [10]:
df.dtypes

country                                 object
iso_code                                object
date                                    object
total_vaccinations                       Int64
people_vaccinated                        Int64
people_fully_vaccinated                  Int64
daily_vaccinations_raw                 float64
daily_vaccinations                       Int64
total_vaccinations_per_hundred         float64
people_vaccinated_per_hundred          float64
people_fully_vaccinated_per_hundred    float64
daily_vaccinations_per_million           Int64
vaccines                                object
source_name                             object
source_website                          object
dtype: object

In [11]:
file = open('file.txt', 'r')
file.readlines()
file.close()

with open('file.txt', 'r') as file:
    file.readlines()



In [12]:
print(pd.io.sql.get_schema(df.reset_index(), 'test.vacinations_per_country', con=source))



CREATE TABLE `test.vacinations_per_country` (
	`index` BIGINT, 
	country TEXT, 
	iso_code TEXT, 
	date TEXT, 
	total_vaccinations BIGINT, 
	people_vaccinated BIGINT, 
	people_fully_vaccinated BIGINT, 
	daily_vaccinations_raw FLOAT(53), 
	daily_vaccinations BIGINT, 
	total_vaccinations_per_hundred FLOAT(53), 
	people_vaccinated_per_hundred FLOAT(53), 
	people_fully_vaccinated_per_hundred FLOAT(53), 
	daily_vaccinations_per_million BIGINT, 
	vaccines TEXT, 
	source_name TEXT, 
	source_website TEXT
)




In [13]:
with source.begin() as transaction:
    df.to_sql('vacinations_per_country', con=transaction, schema='test', if_exists='append', index=False)
    

In [18]:
create = pd.io.sql.get_schema(df.reset_index(), 'test.vacinations_per_country', con=source)
with open('vacinations_per_country.sql', 'w+') as file:
    file.write(create)
    file.close()
    
def populate():
    DDls = 'INSERT INTO test.vacinations_per_country (`'+ str('`, `'.join(df.columns))+ '`)\nVALUES '
    for index, row in df.iterrows():   
        if index == len(df) - 1:
            DDls += str(tuple(row.values)) + ';' 
        else:
            DDls += str(tuple(row.values)) + ',\n'
    return DDls

with open('vacinations_per_country.sql', 'a+') as file:
    file.write(populate())
    file.close()